In [101]:
import pandas as pd
import os
import sqlite3
import pprint
import json
import re

In [98]:
DATA_DIR = os.path.join("..", "data")
ELECCAP_SUFFIX = "eleccap_20220404-201215.csv"
ELECCAP_PATH = os.path.join(DATA_DIR, ELECCAP_SUFFIX)
TABLE_NAME = "eleccap"
DB_PATH = os.path.join(DATA_DIR, "world_eletric_cap.db")

In [67]:
df = pd.read_csv(
    ELECCAP_PATH,
    header=None, 
    names=["country", "tech", "grid_conn_type", "year", "capacity"], # it seems that capacity is in MW and normalized by country area
    sep=";"
)

In [68]:
df.dtypes

country            object
tech               object
grid_conn_type     object
year              float64
capacity           object
dtype: object

In [69]:
df.head(30)

,country,tech,grid_conn_type,year,capacity
0,Afghanistan,On-grid Solar photovoltaic,Off-grid,2011.0,..
1,NaN,NaN,NaN,2012.0,..
2,NaN,NaN,NaN,2013.0,..
3,NaN,NaN,NaN,2014.0,..
4,NaN,NaN,NaN,2015.0,..
5,NaN,NaN,NaN,2016.0,..
6,NaN,NaN,NaN,2017.0,..
7,NaN,NaN,NaN,2018.0,..
8,NaN,NaN,NaN,2019.0,..
9,NaN,NaN,NaN,2020.0,..


In [75]:
df.tail(25)

,country,tech,grid_conn_type,year,capacity
93239,NaN,NaN,NaN,2020.0,..
93240,NaN,NaN,NaN,NaN,NaN
93241,The renewable power capacity data shown in the...,NaN,NaN,NaN,NaN
93242,NaN,NaN,NaN,NaN,NaN
93243,NaN,NaN,NaN,NaN,NaN
93244,Source:,NaN,NaN,NaN,NaN
93245,"IRENA (2021), Renewable capacity statistics 20...",NaN,NaN,NaN,NaN
93246,NaN,NaN,NaN,NaN,NaN
93247,NaN,NaN,NaN,NaN,NaN
93248,NaN,NaN,NaN,NaN,NaN


Precisamos remover as linhas de referência no fim, corrigir esses NaN (replicar o país e a tecnologia até encotrar um valor "não-NaN") e lidar com as capacidades energéticas com valor "..".

In [76]:
df = df.iloc[0:93240]

In [77]:
df.tail()

,country,tech,grid_conn_type,year,capacity
93235,NaN,NaN,NaN,2016.0,..
93236,NaN,NaN,NaN,2017.0,..
93237,NaN,NaN,NaN,2018.0,..
93238,NaN,NaN,NaN,2019.0,..
93239,NaN,NaN,NaN,2020.0,..


In [78]:
# fix string NaNs
df[["country", "tech", "grid_conn_type"]] = df[["country", "tech", "grid_conn_type"]].ffill()

In [79]:
# remove missing ECs
df = df.drop(index=df.loc[df["capacity"] == ".."].index)

In [80]:
df = df.reset_index(drop=True)

In [81]:
df.head(30)

,country,tech,grid_conn_type,year,capacity
0,Afghanistan,On-grid Solar photovoltaic,On-grid,2017.0,"1,70"
1,Afghanistan,On-grid Solar photovoltaic,On-grid,2018.0,"1,70"
2,Afghanistan,On-grid Solar photovoltaic,On-grid,2019.0,"11,70"
3,Afghanistan,On-grid Solar photovoltaic,On-grid,2020.0,"11,70"
4,Afghanistan,Off-grid Solar photovoltaic,Off-grid,2011.0,"0,12"
5,Afghanistan,Off-grid Solar photovoltaic,Off-grid,2012.0,"16,12"
6,Afghanistan,Off-grid Solar photovoltaic,Off-grid,2013.0,"16,84"
7,Afghanistan,Off-grid Solar photovoltaic,Off-grid,2014.0,"18,36"
8,Afghanistan,Off-grid Solar photovoltaic,Off-grid,2015.0,"18,93"
9,Afghanistan,Off-grid Solar photovoltaic,Off-grid,2016.0,"20,17"


In [82]:
df.sample(20)

,country,tech,grid_conn_type,year,capacity
2358,Bulgaria,Nuclear,On-grid,2013.0,"1982,00"
2334,Bulgaria,Solid biofuels,On-grid,2018.0,"33,02"
8328,Italy,Renewable hydropower,On-grid,2016.0,"14991,00"
15908,Sweden,Renewable hydropower,On-grid,2016.0,"16367,00"
3859,Costa Rica,Fossil fuels,On-grid,2018.0,"490,80"
17378,United Republic of Tanzania,Solid biofuels,On-grid,2016.0,"43,40"
16171,Syrian Arab Republic,Biogas,On-grid,2011.0,"6,71"
16244,Tajikistan,Natural gas,On-grid,2017.0,"318,00"
13793,Romania,Concentrated solar power,On-grid,2020.0,"0,09"
185,Algeria,Fossil fuels n.e.s.,On-grid,2017.0,"18892,00"


In [83]:
df.tail()

,country,tech,grid_conn_type,year,capacity
18377,Zimbabwe,Oil,On-grid,2016.0,"10,00"
18378,Zimbabwe,Oil,On-grid,2017.0,"10,00"
18379,Zimbabwe,Oil,On-grid,2018.0,"10,00"
18380,Zimbabwe,Oil,On-grid,2019.0,"10,00"
18381,Zimbabwe,Oil,On-grid,2020.0,"10,00"


In [87]:
df["capacity"] = df["capacity"].map(lambda s: s.replace(",", ".") if isinstance(s, str) else s).astype(float)

In [92]:
df["year"] = df["year"].astype(int)

In [93]:
df.dtypes

country            object
tech               object
grid_conn_type     object
year                int32
capacity          float64
dtype: object

In [94]:
df.sample(20)

,country,tech,grid_conn_type,year,capacity
13660,Republic of North Macedonia,Fossil fuels,On-grid,2014,1077.00
12753,Peru,Biogas,Off-grid,2019,0.00
7739,India,Oil,On-grid,2016,14187.63
7329,Honduras,Off-grid Solar photovoltaic,Off-grid,2011,4.48
7236,Guyana,Fossil fuels,On-grid,2013,201.23
6026,France,Fossil fuels n.e.s.,On-grid,2017,19942.35
8831,Kenya,Onshore wind energy,Off-grid,2020,0.60
17589,United States of America,Fossil fuels,On-grid,2016,750631.00
11920,Niger,Coal and peat,On-grid,2020,37.60
8076,Iraq,Fossil fuels,On-grid,2012,14439.00


In [104]:
df.describe(include="all")

,country,tech,grid_conn_type,year,capacity
count,18382,18382,18382,18382.000000,1.838200e+04
unique,222,21,2,NaN,NaN
top,United States of America,Fossil fuels,On-grid,NaN,NaN
freq,188,2746,14500,NaN,NaN
mean,NaN,NaN,NaN,2015.671309,5.642212e+03
std,NaN,NaN,NaN,2.854219,4.253685e+04
min,NaN,NaN,NaN,2011.000000,0.000000e+00
25%,NaN,NaN,NaN,2013.000000,5.140000e+00
50%,NaN,NaN,NaN,2016.000000,8.500000e+01
75%,NaN,NaN,NaN,2018.000000,1.009575e+03


In [105]:
df["tech"].unique()

array(['On-grid Solar photovoltaic', 'Off-grid Solar photovoltaic',
       'Onshore wind energy', 'Renewable hydropower', 'Fossil fuels',
       'Oil', 'Natural gas', 'Fossil fuels n.e.s.',
       'Renewable municipal waste', 'Other non-renewable energy',
       'Concentrated solar power', 'Solid biofuels', 'Pumped storage',
       'Biogas', 'Nuclear', 'Mixed Hydro Plants', 'Geothermal energy',
       'Marine energy', 'Coal and peat', 'Liquid biofuels',
       'Offshore wind energy'], dtype=object)

Save to DB

In [99]:
con = sqlite3.connect(DB_PATH)
df.to_sql(TABLE_NAME, con=con)

18382

In [102]:
with con:
    cur = con.cursor()
    cur.execute(f"SELECT * FROM {TABLE_NAME} LIMIT 30")
    pprint.pprint(cur.fetchall())

[(0, 'Afghanistan', 'On-grid Solar photovoltaic', 'On-grid', 2017, 1.7),
 (1, 'Afghanistan', 'On-grid Solar photovoltaic', 'On-grid', 2018, 1.7),
 (2, 'Afghanistan', 'On-grid Solar photovoltaic', 'On-grid', 2019, 11.7),
 (3, 'Afghanistan', 'On-grid Solar photovoltaic', 'On-grid', 2020, 11.7),
 (4, 'Afghanistan', 'Off-grid Solar photovoltaic', 'Off-grid', 2011, 0.12),
 (5, 'Afghanistan', 'Off-grid Solar photovoltaic', 'Off-grid', 2012, 16.12),
 (6, 'Afghanistan', 'Off-grid Solar photovoltaic', 'Off-grid', 2013, 16.84),
 (7, 'Afghanistan', 'Off-grid Solar photovoltaic', 'Off-grid', 2014, 18.36),
 (8, 'Afghanistan', 'Off-grid Solar photovoltaic', 'Off-grid', 2015, 18.93),
 (9, 'Afghanistan', 'Off-grid Solar photovoltaic', 'Off-grid', 2016, 20.17),
 (10, 'Afghanistan', 'Off-grid Solar photovoltaic', 'Off-grid', 2017, 20.32),
 (11, 'Afghanistan', 'Off-grid Solar photovoltaic', 'Off-grid', 2018, 19.86),
 (12, 'Afghanistan', 'Off-grid Solar photovoltaic', 'Off-grid', 2019, 19.39),
 (13, 'Afgh

In [103]:
con.close()